# Training Model

In [1]:
!pip install indic-nlp-library -q

ERROR: sphinx 5.3.0 has requirement docutils<0.20,>=0.14, but you'll have docutils 0.20.1 which is incompatible.
You should consider upgrading via the 'c:\users\rampu\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install fasttext_wheel 

You should consider upgrading via the 'c:\users\rampu\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [3]:
import fasttext
import fasttext.util

In [11]:
ft = fasttext.load_model('C:/sai/projects/major project/flask_app/indicnlp.ft.te.300.bin')
ft.get_dimension()

300

In [13]:
fasttext.util.reduce_model(ft, 50)
ft.get_dimension()

50

In [14]:
from indicnlp.tokenize import indic_tokenize
def tokenize_telugu(text):
    return indic_tokenize.trivial_tokenize(text)

In [15]:
import pandas as pd

# Load train and validation data
train_data = pd.read_csv("telugu_train.csv")
val_data = pd.read_csv("telugu_val.csv")
test_data = pd.read_csv("telugu_test.csv")

# Tokenize the text data
train_data['tokens'] = train_data['text'].apply(tokenize_telugu)
val_data['tokens'] = val_data['text'].apply(tokenize_telugu)
test_data['tokens'] = test_data['text'].apply(tokenize_telugu)

In [16]:
def get_sentence_vector(tokens, model):
    # FastText does not have wv attribute, so we use 'get_word_vector'
    vectors = [model.get_word_vector(word) for word in tokens]
    if not vectors:
        return None
    return sum(vectors) / len(vectors)

In [17]:
# Apply the function to generate word embeddings
train_data['embeddings'] = train_data['tokens'].apply(lambda x: get_sentence_vector(x, ft))
val_data['embeddings'] = val_data['tokens'].apply(lambda x: get_sentence_vector(x, ft))
test_data['embeddings'] = test_data['tokens'].apply(lambda x: get_sentence_vector(x, ft))

In [18]:
X_train = list(train_data['embeddings'])
y_train = train_data['label']

X_val = list(val_data['embeddings'])
y_val = val_data['label']

X_test = list(test_data['embeddings'])
y_test = test_data['label']

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [20]:
classifiers = {
#     'Decision Tree': DecisionTreeClassifier(random_state=42),
#     'k-NN': KNeighborsClassifier(),
#     'Naive Bayes': GaussianNB(),
#     'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42)
}

In [21]:
import joblib
import pickle
for name, classifier in classifiers.items():
    print(f"\nTraining and evaluating {name}...")
    classifier.fit(X_train, y_train)

    # Predict on the validation set
    predictions_val = classifier.predict(X_val)

    # Evaluate accuracy on the validation set
    accuracy_val = accuracy_score(y_val, predictions_val)
    print(f"Accuracy on validation set for {name}: {accuracy_val * 100:.2f}%")
    with open('model.pkl', 'wb') as f:
        pickle.dump(classifier, f)


Training and evaluating SVM...
Accuracy on validation set for SVM: 84.52%


In [22]:
# Evaluate each classifier on the test set
for name, classifier in classifiers.items():
    # Predict on the test set
    predictions_test = classifier.predict(X_test)

    # Evaluate accuracy on the test set
    accuracy_test = accuracy_score(y_test, predictions_test)
    print(f"\nAccuracy on test set for {name}: {accuracy_test * 100:.2f}%")


Accuracy on test set for SVM: 83.63%


# Tesing Model

In [24]:
import pickle
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [29]:
text = "hello world"

In [30]:
text_tokens = tokenize_telugu(text)
text_embedding = get_sentence_vector(text_tokens, ft)

In [31]:
prediction = model.predict([text_embedding])
print("Abusive" if prediction == 0 else "Not Abusive")

Not Abusive


In [ ]:
text_embedding

array([ 0.25373834,  0.11429359, -0.46006247,  0.08539951, -0.35152858,
        0.41466865, -0.50430715,  0.23304223,  0.28547463,  0.27407438,
       -0.06326284,  0.01679939,  0.14189519,  0.27268705,  0.07280811,
       -0.33512124, -0.26886582, -0.13984728, -0.25157177,  0.509647  ,
        0.06752215,  0.22076799, -0.09672558,  0.24390587,  0.12136319,
       -0.1530877 , -0.0298101 , -0.16665901, -0.3009723 , -0.03589639,
        0.13174278, -0.08748925, -0.04589874,  0.0035856 , -0.1692615 ,
       -0.1627296 ,  0.09659679, -0.01322322,  0.00985536,  0.10169964,
        0.05615437, -0.1327372 ,  0.00719264, -0.07197587, -0.10012716,
       -0.05117296,  0.02871909,  0.03819418,  0.23590522, -0.17053461],
      dtype=float32)